In [1]:
import csv
import pandas as pd
from datetime import datetime

def load_csv(csv_filename, columns, to_flatten, value):
    with open(csv_filename, 'r', newline='\n') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        
        data = {}
        header = {elt: index for index, elt in enumerate(next(csv_reader))}
        flattens = set()
        
        for row in csv_reader:
            key = tuple(row[header[column]] for column in columns)
            flattens.add(row[header[to_flatten]])
            try:
                data[key][row[header[to_flatten]]] = row[header[value]]
            except KeyError:
                data[key] = {row[header[to_flatten]]: row[header[value]]}
                
        df_dict = {elt:  [] for elt in columns}
        df_dict.update({elt: [] for elt in flattens})
    
        for key, values in data.items():
            for index, column in enumerate(columns):
                df_dict[column].append(key[index])
            for flatten in flattens:
                df_dict[flatten].append(values.get(flatten, 0)) 
        
        return pd.DataFrame(df_dict)

In [2]:
p_columns = ['LIBELLE_ARRET', 'CAT_JOUR', 'ID_REFA_LDA']
p_flatten = 'TRNC_HORR_60'
p_value = 'pourc_validations'

v_columns = ['LIBELLE_ARRET', 'JOUR', 'ID_REFA_LDA']
v_flatten = 'CATEGORIE_TITRE'
v_value = 'NB_VALD'

profile_s1 = load_csv('Data/validations-profils-horaires-2017s1.csv', p_columns, p_flatten, p_value)
profile_s2 = load_csv('Data/validations-profils-horaires-2017s2.csv', p_columns, p_flatten, p_value)

validation_s1 = load_csv('Data/validations-nombre-par-jour-2017s1.csv', v_columns, v_flatten, v_value)
validation_s2 = load_csv('Data/validations-nombre-par-jour-2017s2.csv', v_columns, v_flatten, v_value)

In [3]:
def clean(df, float_columns, not_defined):
    to_return = df.copy()
    
    to_return = to_return.drop(not_defined, axis=1)
    to_return = to_return.drop(to_return[to_return['ID_REFA_LDA'] == ''].index)
    
    for column in float_columns:
        to_return[column] = to_return[column].astype(float)
    
    return to_return

TRC_HORR = [
    '0H-1H', '1H-2H', '2H-3H', '3H-4H', '4H-5H', '5H-6H',
    '6H-7H', '7H-8H', '8H-9H', '9H-10H', '10H-11H', '11H-12H',
    '12H-13H', '13H-14H', '14H-15H', '15H-16H', '16H-17H', '17H-18H',
    '18H-19H', '19H-20H', '20H-21H', '21H-22H', '22H-23H', '23H-0H',
]

CATEGORIE_TITRE = ['AMETHYSTE', 'AUTRE TITRE', 'FGT', 'IMAGINE R', 'NAVIGO', 'TST']

profile_s1 = clean(profile_s1, TRC_HORR, 'ND')
profile_s2 = clean(profile_s2, TRC_HORR, 'ND')

validation_s1 = validation_s1.replace('Moins de 5', 5).replace('', 0)
validation_s2 = validation_s2.replace('Moins de 5', 5).replace('', 0)

validation_s1 = clean(validation_s1, CATEGORIE_TITRE, 'NON DEFINI')
validation_s2 = clean(validation_s2, CATEGORIE_TITRE, 'NON DEFINI')

## Concat profile datasets

In [4]:
def concat_profile(df):
    def apply(row):
        id = row.ID_REFA_LDA
        cat_jour = row.CAT_JOUR
        try:
            new_row = df[(df.ID_REFA_LDA == id) & (df.CAT_JOUR == cat_jour)].iloc[0]
        except:
            new_row = row
            
        for hour in TRC_HORR:
            row[hour] = (row[hour] + new_row[hour]) / 2
        return row
    return apply
    
profile = profile_s2.apply(concat_profile(profile_s1), axis=1)

In [11]:
CATEGORIE_JOUR = ['JOHV', 'SAHV', 'JOVS', 'SAVS', 'DIJFP']

## Concat Validations Number

In [5]:
validation = pd.concat([validation_s1, validation_s2])

# TODO: write this in utils functions
validation['TOTAL_VALID'] = sum(validation[titre] for titre in CATEGORIE_TITRE)

## Adapt validations number

In [6]:
saturday = 6
sunday = 7
week_holydays = [6, 7, 14, 15, 28, 29, 30, 31, 32, 33, 34, 35, 43, 44, 52]
jours_feries = [
    '2017-01-01', '2017-04-16', '2017-04-17', '2017-05-01',
    '2017-05-08', '2017-05-25', '2017-06-05', '2017-07-14',
    '2017-08-15', '2017-11-01', '2017-11-11', '2017-12-25'
]

def get_cat_jour(row):
    if row.JOUR in jours_feries:
        return 'DIJFP'
    
    date = datetime.strptime(row.JOUR, '%Y-%m-%d')
    if date.isocalendar()[1] == sunday:
        return 'DIJFP'
    elif date.isocalendar()[1] == saturday:
        if date.isocalendar()[2] in week_holydays:
            return 'SAVS'
        return 'SAHV'
    
    if date.isocalendar()[2] in week_holydays:
        return 'JOVS'
    return 'JOHV'

validation = pd.concat([validation_s1, validation_s2])
validation['CAT_JOUR'] = validation.apply(get_cat_jour, axis=1)

In [10]:
Data = profile.copy()
for i in range(Data.shape[0]):
    ID_REFA_LDA = Data.iloc[i,25]
    val_day = sum(validation.loc[validation['ID_REFA_LDA'] == ID_REFA_LDA,:].iloc[:, 3])
    for h in range(24):
        Data.iloc[i, h] = int(Data.iloc[i, h] * val_day)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

'68419'